# Machine Learning Engineer Nanodegree Capstone Project
## Forest Type Prediction
## Bin Luo

## Exploring the Data

In [1]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

#import data
import pandas as pd
dataset = pd.read_csv("../data_tree/train.csv")

#drop the ID columns
dataset = dataset.iloc[:,1:]

In [2]:
#data description - shape
r, c = dataset.shape
print (r,c)

(15120, 55)


In [3]:
#data description - statistical
pd.set_option('display.max_columns', None)
dataset.describe()
# Learning :
# No attribute is missing as count is 15120 for all attributes. Hence, all rows can be used
# Wilderness_Area and Soil_Type are one hot encoded. Hence, they could be converted back for some analysis
# Attributes Soil_Type7 and Soil_Type15 can be removed as they are constant
# Scales are not the same for all. Hence, rescaling and standardization may be necessary for some algos

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.0,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.0,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000
mean,2749.322553,156.676653,16.501587,227.195701,51.076521,1714.023214,212.704299,218.965608,135.091997,1511.147288,0.237897,0.033003,0.419907,0.309193,0.023479,0.041204,0.063624,0.055754,0.010913,0.042989,0.0,0.000066,0.000661,0.141667,0.026852,0.015013,0.031481,0.011177,0.0,0.007540,0.040476,0.003968,0.003042,0.009193,0.001058,0.022817,0.050066,0.016997,0.000066,0.003571,0.000992,0.000595,0.085384,0.047950,0.021958,0.045635,0.040741,0.001455,0.006746,0.000661,0.002249,0.048148,0.043452,0.030357,4.000000
std,417.678187,110.085801,8.453927,210.075296,61.239406,1325.066358,30.561287,22.801966,45.895189,1099.936493,0.425810,0.178649,0.493560,0.462176,0.151424,0.198768,0.244091,0.229454,0.103896,0.202840,0.0,0.008133,0.025710,0.348719,0.161656,0.121609,0.174621,0.105133,0.0,0.086506,0.197080,0.062871,0.055075,0.095442,0.032514,0.149326,0.218089,0.129265,0.008133,0.059657,0.031482,0.024391,0.279461,0.213667,0.146550,0.208699,0.197696,0.038118,0.081859,0.025710,0.047368,0.214086,0.203880,0.171574,2.000066
min,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2376.000000,65.000000,10.000000,67.000000,5.000000,764.000000,196.000000,207.000000,106.000000,730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,2752.000000,126.000000,15.000000,180.000000,32.000000,1316.000000,220.000000,223.000000,138.000000,1256.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,3104.00000

In [4]:
#classification distribution
dataset.groupby('Cover_Type').size()

#learning：equally distributed

Cover_Type
1    2160
2    2160
3    2160
4    2160
5    2160
6    2160
7    2160
dtype: int64

### Data correlation

In [5]:
import numpy
# Correlation requires continous data. Hence, ignore Wilderness_Area and Soil_Type as they are binary
size = 10
data_corr = dataset.iloc[:,:size]

cols = data_corr.columns

# Calculates pearson co-efficient for all combinations
data_corr_coeff = data_corr.corr()

threshold = 0.5

high_corr_list = []

for i in range(0,size):
    for j in range(i+1, size):
        if(data_corr_coeff.iloc[i,j]>=threshold or data_corr_coeff.iloc[i,j]<=-threshold):
            high_corr_list.append([data_corr_coeff.iloc[i,j],i,j])
s_high_corr_list = sorted(high_corr_list,key=lambda x: -abs(x[0]))
for v, i, j in s_high_corr_list:
    print ("%s and %s co-efficent is %.2f" % (cols[i],cols[j],v))

Hillshade_9am and Hillshade_3pm co-efficent is -0.78
Horizontal_Distance_To_Hydrology and Vertical_Distance_To_Hydrology co-efficent is 0.65
Aspect and Hillshade_3pm co-efficent is 0.64
Hillshade_Noon and Hillshade_3pm co-efficent is 0.61
Slope and Hillshade_Noon co-efficent is -0.61
Aspect and Hillshade_9am co-efficent is -0.59
Elevation and Horizontal_Distance_To_Roadways co-efficent is 0.58


### Data Cleaning

In [6]:
#remove the constant columns

const = []

for col in dataset.columns:
    if dataset[col].std()==0:
        const.append(col)

dataset.drop(const,axis=1,inplace=True)

print(const)

['Soil_Type7', 'Soil_Type15']


## Data Preparation

In [7]:
#obtain new # of row and column
r, c = dataset.shape

#get columns' name
cols = dataset.columns

array = dataset.values

X = array[:,0:(c-1)]
Y = array[:,(c-1)]

val_size = 0.1

seed = 42

#Split the data into chunks
from sklearn import cross_validation
X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(X, Y, test_size=val_size,random_state=seed)

#Import libraries for data transformations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

X_all = []
X_all.append(['Orig','All',X_train,X_val])

#size of the non-categorical data
size = 10

#standardized
X_temp = StandardScaler().fit_transform(X_train[:,0:size])
X_val_temp = StandardScaler().fit_transform(X_val[:,0:size])
#Concatenate non-categorical data and categorical
X_con = numpy.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = numpy.concatenate((X_val_temp,X_val[:,size:]),axis=1)
#Add this version of X to the list 
X_all.append(['StdSca','All', X_con,X_val_con])

#MinMax
#Apply transform only for non-categorical data
X_temp = MinMaxScaler().fit_transform(X_train[:,0:size])
X_val_temp = MinMaxScaler().fit_transform(X_val[:,0:size])
#Concatenate non-categorical data and categorical
X_con = numpy.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = numpy.concatenate((X_val_temp,X_val[:,size:]),axis=1)
#Add this version of X to the list 
X_all.append(['MinMax', 'All', X_con,X_val_con])

#Normalize
#Apply transform only for non-categorical data
X_temp = Normalizer().fit_transform(X_train[:,0:size])
X_val_temp = Normalizer().fit_transform(X_val[:,0:size])
#Concatenate non-categorical data and categorical
X_con = numpy.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = numpy.concatenate((X_val_temp,X_val[:,size:]),axis=1)
#Add this version of X to the list 
X_all.append(['Norm', 'All', X_con,X_val_con])

#List of transformations
transform_list = []

for trans,name,X,X_val in X_all:
    transform_list.append(trans)


/Users/binluo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/binluo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/binluo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/binluo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/binluo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: D

## Feature Selection - based on feature importance
* ExtraTreesClassifier
* RandomForestClassifier

In [8]:
#Select top 75%
ratios = [0.75,0.5]

#Import the libraries
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

models = []
models.append(['Extra',ExtraTreesClassifier(n_estimators=c-1,n_jobs=-1,random_state=seed)])
models.append(['RndFst',RandomForestClassifier(n_estimators=c-1,n_jobs=-1,random_state=seed)])

X_select = []

#for all transformation of X_all
for trans, p, X, X_val in X_all:
    #for all feature importance based selection models
    for name, model in models:
        #train model to get feature importance
        model.fit(X,Y_train)
        # sort & store feature importance
        feat_imp = []
        for i, imp in enumerate(list(model.feature_importances_)):
            feat_imp.append([i,cols[i],imp])
        
        imp_sorted = sorted(feat_imp,key=lambda x: -x[2])
        
        #loop through different ratio
        for ratio in ratios:
            #store selected columns based on ratio
            remove_start = int((ratio*(c-1)))
            #store ranking for each feature
            ranking = []
            select_list = []
            remove_list = []
            for j, (i,col,x) in enumerate(list(imp_sorted)):
                ranking.append([i,cols[i],j])
                if (j<remove_start):
                    select_list.append([i,col,j])
                else:
                    remove_list.append([i,col,j])
            ranking_sorted = sorted(ranking,key=lambda x: x[0])
            X_select.append([trans,name,ratio,[x[2] for x in ranking_sorted],select_list,remove_list])

## Feature Selection - RFE, based on feature ranking

In [9]:
#Libraries for feature selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

LG_model = LogisticRegression(random_state=seed,n_jobs=-1)
RFE_models=[]
RFE_models.append(['RFE_0.75',RFE(LG_model,0.75*(c-1)),0.75])
RFE_models.append(['RFE_0.5',RFE(LG_model,0.5*(c-1)),0.5])

#for all transformation of X_all
for trans, p, X, X_val in X_all:
    #for all feature importance based selection models
    for name, model,ratio in RFE_models:
        #train model to get feature importance
        model.fit(X,Y_train)
        # sort & store feature importance
        feat_imp = []
        for i, imp in enumerate(list(model.ranking_)):
            feat_imp.append([i,cols[i],imp])
        
        imp_sorted = sorted(feat_imp,key=lambda x: -x[2])
        
        #remove starting point
        remove_start = int((ratio*(c-1)))
        select_list = []
        remove_list = []
        for j, (i,col,x) in enumerate(list(imp_sorted)):

            if (j<remove_start):
                select_list.append([i,col,j])
            else:
                remove_list.append([i,col,j])

        X_select.append([trans,name,ratio,[x[2] for x in feat_imp],select_list,remove_list])
#print X_select    

In [ ]:
#Import plotting library    
import matplotlib.pyplot as plt 

rank_summary = pd.DataFrame(data=[x[3] for x in X_select],columns=cols[:c-1])
rank_median = rank_summary.median()
_ = rank_summary.boxplot(rot=90)
plt.show()

## Prediction, Evaluation & Analysis
* LDA - linear algo

In [11]:
#create a list to store all the all top-performance combination across all the tested algorithms 
best_algo = []

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()
#model = LogisticRegression(n_jobs=-1,random_state=seed,C=100)
algo_name = 'LDA'

#placeholder to store the combination with best performance for this algorithm
best_result = ['placeholder',0.0]

#get the performances of using all features and different transformed data to train the model
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features and different transformed data to train the model
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+r
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

Orig + All, 0.661376
StdSca + All, 0.659392
MinMax + All, 0.625000
Norm + All, 0.651455
Orig + Extra, #of used features is 39, accuracy is 0.657407
Orig + Extra, #of used features is 26, accuracy is 0.638228
Orig + RndFst, #of used features is 39, accuracy is 0.659392
Orig + RndFst, #of used features is 26, accuracy is 0.646825
StdSca + Extra, #of used features is 39, accuracy is 0.658069
StdSca + Extra, #of used features is 26, accuracy is 0.638889
StdSca + RndFst, #of used features is 39, accuracy is 0.658730
StdSca + RndFst, #of used features is 26, accuracy is 0.646825
MinMax + Extra, #of used features is 39, accuracy is 0.619709
MinMax + Extra, #of used features is 26, accuracy is 0.606481
MinMax + RndFst, #of used features is 39, accuracy is 0.621032
MinMax + RndFst, #of used features is 26, accuracy is 0.607804
Norm + Extra, #of used features is 39, accuracy is 0.648148
Norm + Extra, #of used features is 26, accuracy is 0.628968
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* LR - logistic regression

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(n_jobs=-1,random_state=seed,C=100)
algo_name = 'LR'

best_result = ['placeholder',0.0]
#performances of using all features and different transformed data to train the model
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features and different transformed data to train the model
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+r
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

#training time is longer than LDA

Orig + All, 0.679233
StdSca + All, 0.693783
MinMax + All, 0.639550
Norm + All, 0.668651
Orig + Extra, #of used features is 39, accuracy is 0.672619
Orig + Extra, #of used features is 26, accuracy is 0.660053
Orig + RndFst, #of used features is 39, accuracy is 0.663360
Orig + RndFst, #of used features is 26, accuracy is 0.654762
StdSca + Extra, #of used features is 39, accuracy is 0.686508
StdSca + Extra, #of used features is 26, accuracy is 0.658069
StdSca + RndFst, #of used features is 39, accuracy is 0.685185
StdSca + RndFst, #of used features is 26, accuracy is 0.658730
MinMax + Extra, #of used features is 39, accuracy is 0.640873
MinMax + Extra, #of used features is 26, accuracy is 0.631614
MinMax + RndFst, #of used features is 39, accuracy is 0.640873
MinMax + RndFst, #of used features is 26, accuracy is 0.636243
Norm + Extra, #of used features is 39, accuracy is 0.660714
Norm + Extra, #of used features is 26, accuracy is 0.635582
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* KNN - Non linear, K-nearest neighbors

In [13]:
from sklearn.neighbors import KNeighborsClassifier

#creat model with 1 neighbor
model = KNeighborsClassifier(n_jobs=-1,n_neighbors=1)
algo_name = 'KNN'

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+r
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

Orig + All, 0.857804
StdSca + All, 0.822090
MinMax + All, 0.812169
Norm + All, 0.873677
Orig + Extra, #of used features is 39, accuracy is 0.857804
Orig + Extra, #of used features is 26, accuracy is 0.857804
Orig + RndFst, #of used features is 39, accuracy is 0.857804
Orig + RndFst, #of used features is 26, accuracy is 0.857804
StdSca + Extra, #of used features is 39, accuracy is 0.822090
StdSca + Extra, #of used features is 26, accuracy is 0.816138
StdSca + RndFst, #of used features is 39, accuracy is 0.822090
StdSca + RndFst, #of used features is 26, accuracy is 0.818122
MinMax + Extra, #of used features is 39, accuracy is 0.812169
MinMax + Extra, #of used features is 26, accuracy is 0.798280
MinMax + RndFst, #of used features is 39, accuracy is 0.812169
MinMax + RndFst, #of used features is 26, accuracy is 0.802249
Norm + Extra, #of used features is 39, accuracy is 0.869709
Norm + Extra, #of used features is 26, accuracy is 0.858466
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* CART - Non-linear classifier

In [14]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=seed,max_depth=13)
algo_name = 'CART'

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+ str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

Orig + All, 0.784392
StdSca + All, 0.775794
MinMax + All, 0.699074
Norm + All, 0.710979
Orig + Extra, #of used features is 39, accuracy is 0.782407
Orig + Extra, #of used features is 26, accuracy is 0.781746
Orig + RndFst, #of used features is 39, accuracy is 0.784392
Orig + RndFst, #of used features is 26, accuracy is 0.777778
StdSca + Extra, #of used features is 39, accuracy is 0.773810
StdSca + Extra, #of used features is 26, accuracy is 0.771164
StdSca + RndFst, #of used features is 39, accuracy is 0.773810
StdSca + RndFst, #of used features is 26, accuracy is 0.775132
MinMax + Extra, #of used features is 39, accuracy is 0.693783
MinMax + Extra, #of used features is 26, accuracy is 0.667989
MinMax + RndFst, #of used features is 39, accuracy is 0.687169
MinMax + RndFst, #of used features is 26, accuracy is 0.674603
Norm + Extra, #of used features is 39, accuracy is 0.709656
Norm + Extra, #of used features is 26, accuracy is 0.706349
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* SVM - Non-linear classifier

In [15]:
from sklearn.svm import SVC

model = SVC(random_state=seed,C=10)

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

#Training time is very long compared to other algorithms
#performance is very poor for Original data, data preprocessing is needed

Orig + All, 0.140212
StdSca + All, 0.792328
MinMax + All, 0.700397
Norm + All, 0.629630
Orig + Extra, #of used features is 39, accuracy is 0.137566
Orig + Extra, #of used features is 26, accuracy is 0.136243
Orig + RndFst, #of used features is 39, accuracy is 0.137566
Orig + RndFst, #of used features is 26, accuracy is 0.136243
StdSca + Extra, #of used features is 39, accuracy is 0.794974
StdSca + Extra, #of used features is 26, accuracy is 0.792989
StdSca + RndFst, #of used features is 39, accuracy is 0.794312
StdSca + RndFst, #of used features is 26, accuracy is 0.794974
MinMax + Extra, #of used features is 39, accuracy is 0.703704
MinMax + Extra, #of used features is 26, accuracy is 0.697751
MinMax + RndFst, #of used features is 39, accuracy is 0.705026
MinMax + RndFst, #of used features is 26, accuracy is 0.694444
Norm + Extra, #of used features is 39, accuracy is 0.626323
Norm + Extra, #of used features is 26, accuracy is 0.619048
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* RandomForest - bagging

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=-1,n_estimators=100, random_state=seed)
algo_name = "RndFst"

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

Orig + All, 0.876984
StdSca + All, 0.871032
MinMax + All, 0.833995
Norm + All, 0.869048
Orig + Extra, #of used features is 39, accuracy is 0.880291
Orig + Extra, #of used features is 26, accuracy is 0.867725
Orig + RndFst, #of used features is 39, accuracy is 0.874339
Orig + RndFst, #of used features is 26, accuracy is 0.871693
StdSca + Extra, #of used features is 39, accuracy is 0.880952
StdSca + Extra, #of used features is 26, accuracy is 0.868386
StdSca + RndFst, #of used features is 39, accuracy is 0.879630
StdSca + RndFst, #of used features is 26, accuracy is 0.878307
MinMax + Extra, #of used features is 39, accuracy is 0.822751
MinMax + Extra, #of used features is 26, accuracy is 0.808862
MinMax + RndFst, #of used features is 39, accuracy is 0.831349
MinMax + RndFst, #of used features is 26, accuracy is 0.807540
Norm + Extra, #of used features is 39, accuracy is 0.869709
Norm + Extra, #of used features is 26, accuracy is 0.869709
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* Extra Trees - bagging

In [17]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_jobs=-1,n_estimators=100, random_state=seed)
algo_name = 'ET'

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)


Orig + All, 0.882275
StdSca + All, 0.885582
MinMax + All, 0.849206
Norm + All, 0.873677
Orig + Extra, #of used features is 39, accuracy is 0.879630
Orig + Extra, #of used features is 26, accuracy is 0.876323
Orig + RndFst, #of used features is 39, accuracy is 0.875661
Orig + RndFst, #of used features is 26, accuracy is 0.880291
StdSca + Extra, #of used features is 39, accuracy is 0.876984
StdSca + Extra, #of used features is 26, accuracy is 0.877646
StdSca + RndFst, #of used features is 39, accuracy is 0.884921
StdSca + RndFst, #of used features is 26, accuracy is 0.875661
MinMax + Extra, #of used features is 39, accuracy is 0.843254
MinMax + Extra, #of used features is 26, accuracy is 0.824074
MinMax + RndFst, #of used features is 39, accuracy is 0.845899
MinMax + RndFst, #of used features is 26, accuracy is 0.827381
Norm + Extra, #of used features is 39, accuracy is 0.879630
Norm + Extra, #of used features is 26, accuracy is 0.872354
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* AdaBoost - Boosting

In [18]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=100, random_state=seed)
algo_name = "AdaBoost"

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

Orig + All, 0.388889
StdSca + All, 0.388228
MinMax + All, 0.385582
Norm + All, 0.362434
Orig + Extra, #of used features is 39, accuracy is 0.388889
Orig + Extra, #of used features is 26, accuracy is 0.388889
Orig + RndFst, #of used features is 39, accuracy is 0.388889
Orig + RndFst, #of used features is 26, accuracy is 0.388889
StdSca + Extra, #of used features is 39, accuracy is 0.388228
StdSca + Extra, #of used features is 26, accuracy is 0.388228
StdSca + RndFst, #of used features is 39, accuracy is 0.388228
StdSca + RndFst, #of used features is 26, accuracy is 0.388228
MinMax + Extra, #of used features is 39, accuracy is 0.385582
MinMax + Extra, #of used features is 26, accuracy is 0.385582
MinMax + RndFst, #of used features is 39, accuracy is 0.385582
MinMax + RndFst, #of used features is 26, accuracy is 0.385582
Norm + Extra, #of used features is 39, accuracy is 0.420635
Norm + Extra, #of used features is 26, accuracy is 0.377646
Norm + RndFst, #of used features is 39, accuracy i

## Prediction, Evaluation & Analysis
* GradientBoostng - Boosting

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(max_depth=9, random_state=seed)
algo_name = 'GB'

best_result = ['placeholder',0.0]
#performances of using all features
for trans, p, x_t, x_v in X_all:
    model.fit(x_t,Y_train)
    result = model.score(x_v,Y_val)
    if (result > best_result[1]):
        best_result[0] = algo_name+'_'+trans+'_'+p+'_1.00'
        best_result[1] = result
    print(trans+" + "+p+", %f") % result
    
#performance of using subset of the features 
for trans, name, r, rank, select_list,remove_list in X_select:
    #get the indexes of the features selected 
    i_cols_list = [x[0] for x in select_list]
    #get the tranformed data based for different
    for t, n, x_t, x_v in X_all:
        if (trans==t):
            model.fit(x_t[:,i_cols_list],Y_train)
            result = model.score(x_v[:,i_cols_list], Y_val)
            if (result > best_result[1]):
                best_result[0] = algo_name+'_'+trans+'_'+name+'_'+str(r)
                best_result[1] = result
    print(trans+" + "+name+", #of used features is %d, accuracy is %f" % (r*(c-1),result))
best_algo.append(best_result)
print(best_result)

#training time is extremely high, though the performance is good, closed to 88%

Orig + All, 0.882275
StdSca + All, 0.876323
MinMax + All, 0.788360
Norm + All, 0.873677
Orig + Extra, #of used features is 39, accuracy is 0.876984
Orig + Extra, #of used features is 26, accuracy is 0.868386
Orig + RndFst, #of used features is 39, accuracy is 0.883598
Orig + RndFst, #of used features is 26, accuracy is 0.861772
StdSca + Extra, #of used features is 39, accuracy is 0.871693
StdSca + Extra, #of used features is 26, accuracy is 0.870370
StdSca + RndFst, #of used features is 39, accuracy is 0.876323
StdSca + RndFst, #of used features is 26, accuracy is 0.862434
MinMax + Extra, #of used features is 39, accuracy is 0.787698
MinMax + Extra, #of used features is 26, accuracy is 0.781746
MinMax + RndFst, #of used features is 39, accuracy is 0.785714
MinMax + RndFst, #of used features is 26, accuracy is 0.781085
Norm + Extra, #of used features is 39, accuracy is 0.875000
Norm + Extra, #of used features is 26, accuracy is 0.862434
Norm + RndFst, #of used features is 39, accuracy i

In [218]:
print(best_algo)

[['Orig_All_1.00', 0.66137566137566139], ['StdSca_All_1.00', 0.69378306878306883], ['Norm_All_1.00', 0.87367724867724872], ['Orig_RndFst_0.75', 0.78637566137566139], ['StdSca_Extra_0.5', 0.794973544973545], ['Orig_Extra_0.75', 0.87896825396825395], ['StdSca_All_1.00', 0.88558201058201058], ['Norm_Extra_0.75', 0.42063492063492064], ['Orig_All_1.00', 0.88227513227513232], ['LDA_Orig_All_1.00', 0.66137566137566139], ['LR_StdSca_All_1.00', 0.69378306878306883], ['KNN_Norm_All_1.00', 0.87367724867724872], ['CART_Orig_RndFst_0.75', 0.78637566137566139], ['CART_StdSca_Extra_0.5', 0.794973544973545], ['RndFst_Orig_Extra_0.75', 0.87896825396825395], ['ET_StdSca_All_1.00', 0.88558201058201058], ['AdaBoost_Norm_Extra_0.75', 0.42063492063492064], ['GB_Orig_All_1.00', 0.88227513227513232]]


In [ ]:
#create a plot to identify the algorith with highest performance score 
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
objects = [x[0] for x in best_algo]
y_pos = np.arange(len(objects))
performance = [x[1] for x in best_algo]
 
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Accuracy on Testing data')
plt.title('Model+Feature#+Transformation selection')
 
plt.show()

## Make Prediction with best model - 1st submission

In [ ]:
best_model_pred = ExtraTreesClassifier(n_jobs=-1,n_estimators=100)

#read the test dataset
dataset_test = pd.read_csv("../data_tree/test.csv")
ID = dataset_test['Id']
dataset_test.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)
X_test_pred = dataset_test.values


#performances of using all features
for trans, p, x_t, x_v in X_all:
    if (trans =='StdSca'):
        best_model_pred.fit(x_t,Y_train)

#standardized
X_pred_temp = StandardScaler().fit_transform(X_test_pred[:,0:size])
#Concatenate non-categorical data and categorical
X_test_pred_con = numpy.concatenate((X_pred_temp,X_test_pred[:,size:]),axis=1)
#Add this version of X to the list 
predictions = best_model_pred.predict(X_test_pred_con)

#writing the prediction result for submission
with open("submission_StdSca.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(predictions)):
        subfile.write("%s,%s\n"%(ID[i],pred))

## Feature Engineering - 2nd submission

In [1]:
import pandas as pd
import numpy
from sklearn.ensemble import ExtraTreesClassifier

best_model_sub = ExtraTreesClassifier(n_jobs=-1,n_estimators=100)

#prepare training data
dataset = pd.read_csv("../data_tree/train.csv")
#ID = dataset['Id']
dataset.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)

#create a functiont to do the feature engineering
def feature_engineering(dataset):
    dataset_FE = dataset
    dataset_FE['Ele_minus_VDtHyd'] = dataset['Elevation'] - dataset['Vertical_Distance_To_Hydrology']
    dataset_FE['Ele_plus_VDtHyd'] = dataset['Elevation'] + dataset['Vertical_Distance_To_Hydrology']
    dataset_FE['Distance_to_Hydrolody'] = (dataset['Horizontal_Distance_To_Hydrology']**2 + dataset['Vertical_Distance_To_Hydrology']**2)**0.5
    dataset_FE['Hydro_plus_Fire'] = dataset['Horizontal_Distance_To_Hydrology'] + dataset['Horizontal_Distance_To_Fire_Points']
    dataset_FE['Hydro_minus_Fire'] = dataset['Horizontal_Distance_To_Hydrology'] - dataset['Horizontal_Distance_To_Fire_Points']

    dataset_FE['Hydro_plus_Road'] = dataset['Horizontal_Distance_To_Hydrology'] + dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Hydro_minus_Road'] = dataset['Horizontal_Distance_To_Hydrology'] - dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Fire_plus_Road'] = dataset['Horizontal_Distance_To_Fire_Points'] + dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Fire_minus_Road'] = dataset['Horizontal_Distance_To_Fire_Points'] - dataset['Horizontal_Distance_To_Roadways']
    return dataset_FE


dataset_FE=feature_engineering(dataset)
#dataset_FE.head()

cols = []
cols = list(dataset_FE.columns)
cols.remove('Cover_Type')
X_train_sub = dataset_FE[cols].values
Y_train_sub = dataset_FE['Cover_Type'].values

print Y_train_sub

best_model_sub.fit(X_train_sub,Y_train_sub)


[5 5 2 ..., 3 3 3]


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [2]:
#prepare testing data
#prepare training data
dataset_t = pd.read_csv("../data_tree/test.csv")
ID = dataset_t['Id']
dataset_t.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)


dataset_FE_t=feature_engineering(dataset_t)
X_test_sub = dataset_FE_t.values

predictions = best_model_sub.predict(X_test_sub)

#writing the prediction result for submission
with open("submission_FE2.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(predictions)):
        subfile.write("%s,%s\n"%(ID[i],pred))


## Feature parameter tuning - 3rd submission

In [16]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import f1_score


# Create the parameters list you wish to tune

parameters = {
    'n_estimators': [150],
    'max_features': ['auto',0.6,0.8],
    'min_samples_split': [2,4,8,16,32,64],
    'min_samples_leaf' : [1,10,25,50],
    'n_jobs': [-1],
    'random_state': [42]
}

# Initialize the classifier
best_model_GV = ExtraTreesClassifier()

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,average='weighted')


# create cross-validation data set and apply GridSearch
from sklearn.cross_validation import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(Y_train_sub,10,random_state=42)

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(best_model_GV, parameters, f1_scorer, cv = cv)


# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train_sub, Y_train_sub)

# Get the estimator
clf = grid_obj.best_estimator_


In [17]:
#prepare testing data

dataset_t = pd.read_csv("../data_tree/test.csv")
ID = dataset_t['Id']
dataset_t.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)


dataset_FE_t=feature_engineering(dataset_t)
X_test_sub = dataset_FE_t.values

clf_predictions = clf.predict(X_test_sub)

#writing prediction result for subsmission
with open("submission_clf2.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(clf_predictions)):
        subfile.write("%s,%s\n"%(ID[i],pred))

In [13]:
best_estimator_n_estimator_optimized = grid_obj.best_params_
print(best_estimator_n_estimator_optimized)

{'n_estimators': 150, 'random_state': 42, 'n_jobs': -1}


In [19]:
best_estimator_all_optimized = grid_obj.best_params_
print(best_estimator_all_optimized)

{'n_jobs': -1, 'min_samples_leaf': 1, 'n_estimators': 150, 'min_samples_split': 2, 'random_state': 42, 'max_features': 0.6}


In [14]:
from sklearn.multiclass import OneVsRestClassifier

clf_predictions_OvA=OneVsRestClassifier(ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=1)).fit(X_train_sub, Y_train_sub).predict(dataset_t)

In [15]:
with open("submission_clf_ova.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(clf_predictions_OvA)):
        subfile.write("%s,%s\n"%(ID[i],pred))

## One-VS-all implmentation - 4th submission

In [3]:
#####get training data

import pandas as pd
import numpy
from sklearn.ensemble import ExtraTreesClassifier

#prepare training data
dataset = pd.read_csv("../data_tree/train.csv")
#ID = dataset['Id']
dataset.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)


#add features
def feature_engineering(dataset):
    dataset_FE = dataset
    dataset_FE['Ele_minus_VDtHyd'] = dataset['Elevation'] - dataset['Vertical_Distance_To_Hydrology']
    dataset_FE['Ele_plus_VDtHyd'] = dataset['Elevation'] + dataset['Vertical_Distance_To_Hydrology']
    dataset_FE['Distance_to_Hydrolody'] = (dataset['Horizontal_Distance_To_Hydrology']**2 + dataset['Vertical_Distance_To_Hydrology']**2)**0.5
    dataset_FE['Hydro_plus_Fire'] = dataset['Horizontal_Distance_To_Hydrology'] + dataset['Horizontal_Distance_To_Fire_Points']
    dataset_FE['Hydro_minus_Fire'] = dataset['Horizontal_Distance_To_Hydrology'] - dataset['Horizontal_Distance_To_Fire_Points']

    dataset_FE['Hydro_plus_Road'] = dataset['Horizontal_Distance_To_Hydrology'] + dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Hydro_minus_Road'] = dataset['Horizontal_Distance_To_Hydrology'] - dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Fire_plus_Road'] = dataset['Horizontal_Distance_To_Fire_Points'] + dataset['Horizontal_Distance_To_Roadways']
    dataset_FE['Fire_minus_Road'] = dataset['Horizontal_Distance_To_Fire_Points'] - dataset['Horizontal_Distance_To_Roadways']
    return dataset_FE


dataset_FE=feature_engineering(dataset)
#dataset_FE.head()

cols = []
cols = list(dataset_FE.columns)
cols.remove('Cover_Type')
X_train_sub = dataset_FE[cols].values
Y_train_sub = dataset_FE['Cover_Type'].values


In [4]:
####prep testing data

dataset_t = pd.read_csv("../data_tree/test.csv")
ID = dataset_t['Id']
dataset_t.drop(['Id','Soil_Type7', 'Soil_Type15'],axis=1,inplace=True)


dataset_FE_t=feature_engineering(dataset_t)
X_test_sub = dataset_FE_t.values

In [5]:
#prep training and testing dataset
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_sub, Y_train_sub, test_size=0.1, random_state=42)

In [6]:
#create best model with tuned parameters
from sklearn.metrics import f1_score
best_model_GV_OVA1 = ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=-1)
#best_model_GV_OVA2 = ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=-1,min_samples_leaf=1,min_samples_split=2,max_features=0.6)

pred1 = best_model_GV_OVA1.fit(X_train,y_train).predict(X_test)
#pred2 = best_model_GV_OVA2.fit(X_train,y_train).predict(X_test)

print f1_score(y_test,pred1,average='weighted')
#print f1_score(y_test,pred2,average='weighted')


0.914038809898


In [7]:
#plot confusion matrix to identify the classes was mis-classfied most

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np

class_names = [1,2,3,4,5,6,7]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, pred1)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[186  28   0   0   5   0   8]
 [ 26 173   4   0   6   1   0]
 [  0   1 214   4   0  18   0]
 [  0   0   2 200   0   2   0]
 [  0   1   4   0 198   0   0]
 [  0   1   9   4   0 193   0]
 [  5   0   0   0   0   0 219]]


/Users/binluo/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


Normalized confusion matrix
[[ 0.82  0.12  0.    0.    0.02  0.    0.04]
 [ 0.12  0.82  0.02  0.    0.03  0.    0.  ]
 [ 0.    0.    0.9   0.02  0.    0.08  0.  ]
 [ 0.    0.    0.01  0.98  0.    0.01  0.  ]
 [ 0.    0.    0.02  0.    0.98  0.    0.  ]
 [ 0.    0.    0.04  0.02  0.    0.93  0.  ]
 [ 0.02  0.    0.    0.    0.    0.    0.98]]


In [10]:
#prep list columns name for later usage
cols_OVA = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distance_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire', 'Hydro_plus_Road', 'Hydro_minus_Road', 'Fire_plus_Road', 'Fire_minus_Road', 'Cover_Type']
cols_OVA = list(cols_OVA)
print cols_OVA[0:61]

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distance_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire', 'Hydro_plus_Road', 'Hydro_m

### Predict Type 7

In [11]:
#complete dataset to train but binarize the label
dataset_OVA = dataset_FE[cols_OVA]
df_7 = dataset_OVA
df_7.loc[df_7['Cover_Type'] != 7, 'Cover_Type'] = -1

In [12]:
print(df_7.groupby('Cover_Type').size())

Cover_Type
-1    12960
 7     2160
dtype: int64


In [13]:
#seperate training and testing datasets
X_train_OVA_7 = df_7.iloc[:,0:61].values
y_train_OVA_7 = df_7.iloc[:,61].values
print len(y_train_OVA_7)
print X_train_OVA_7.shape

15120
(15120, 61)


In [14]:
pred_OVA_7 = best_model_GV_OVA1.fit(X_train_OVA_7,y_train_OVA_7).predict(X_test_sub)

In [48]:
###store prediction result for class 7
Result_OVA= pd.DataFrame(
    {'ID': ID.values,
     'pred': pred_OVA_7
    })
print Result_OVA.groupby('pred').size()

#writing out prediction data into csv file for feature consolidation
with open("Result_OVA_7.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(pred_OVA_7)):
        subfile.write("%s,%s\n"%(ID[i],pred))

pred
-1    539154
 7     26738
dtype: int64


### Predict type 6

In [16]:
#to predict class 6, exclude instances with label as class 7

dataset_OVA = dataset_FE[cols_OVA]
df_6 = dataset_OVA.loc[dataset_OVA['Cover_Type']!=7]
print df_6.groupby('Cover_Type').size()

#make all other classes (except for class 6) label as -1
df_6.loc[df_6['Cover_Type'] != 6, 'Cover_Type'] = -1
print df_6.groupby('Cover_Type').size()

#create  training and testing data set
X_train_OVA_6 = df_6.iloc[:,0:61].values
y_train_OVA_6 = df_6.iloc[:,61].values
print len(y_train_OVA_6)
print X_train_OVA_6.shape

###train without instances labeled as 7

best_model_GV_OVA1.fit(X_train_OVA_6,y_train_OVA_6)

Cover_Type
1    2160
2    2160
3    2160
4    2160
5    2160
6    2160
dtype: int64
Cover_Type
-1    10800
 6     2160
dtype: int64
12960
(12960, 61)


/Users/binluo/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [33]:
###predict without labeled as 7
###exclude data labeled as 7 out from the testing
X_test_sub_OVA = dataset_FE_t[cols_OVA[0:61]]
X_test_sub_OVA['ID']=pd.DataFrame({'ID':ID})
print(ID.shape)
print(dataset_FE_t[cols_OVA[0:61]].shape)
print(X_test_sub_OVA.shape)

excl_list = Result_OVA.loc[Result_OVA['pred']!=7,'ID']
excl_list = list(excl_list.values)
X_test_sub_OVA = X_test_sub_OVA[X_test_sub_OVA['ID'].isin(excl_list)]
X_test_sub_OVA.shape

(565892,)
(565892, 61)
(565892, 62)


(539154, 62)

In [34]:
#predict type 6
pred_OVA_6 = best_model_GV_OVA1.predict(X_test_sub_OVA[cols_OVA[0:61]])

In [63]:
##store the prediction result into a dataframe
Result_OVA_6= pd.DataFrame(
    {'ID': X_test_sub_OVA['ID'].values,
     'pred': pred_OVA_6
    })
print Result_OVA_6.groupby('pred').size()

pred
-1    520134
 6     19020
dtype: int64


In [56]:
### store predictionr esult
Result_OVA_6= pd.DataFrame(
    {'ID': X_test_sub_OVA['ID'].values,
     'pred': pred_OVA_6
    })
print Result_OVA_6.groupby('pred').size()

ID_6 = X_test_sub_OVA['ID'].values

print(ID_6.shape)

print(len(pred_OVA_6))

#write the prediction results out to a csv file for future consolidation
with open("Result_OVA_6.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(pred_OVA_6)):
        subfile.write("%s,%s\n"%(ID_6[i],pred))

pred
-1    520134
 6     19020
dtype: int64
(539154,)
539154


### Predict for type 5

In [65]:
####repeate the process - train the model for type 5 excluding type 7 and 6 in the training data
dataset_OVA = dataset_FE[cols_OVA]
df_5 = dataset_OVA.loc[dataset_OVA['Cover_Type']!=7]
df_5 = df_5.loc[df_5['Cover_Type']!=6]
print df_5.groupby('Cover_Type').size()

###make other classes to be -1 lable
df_5.loc[df_5['Cover_Type'] != 5, 'Cover_Type'] = -1
print df_5.groupby('Cover_Type').size()

X_train_OVA_5 = df_5.iloc[:,0:61].values
y_train_OVA_5 = df_5.iloc[:,61].values
print len(y_train_OVA_5)
print X_train_OVA_5.shape

###train without instances labeled as 7 and 6
best_model_GV_OVA5 = ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=-1)
best_model_GV_OVA5.fit(X_train_OVA_5,y_train_OVA_5)

Cover_Type
1    2160
2    2160
3    2160
4    2160
5    2160
dtype: int64
Cover_Type
-1    8640
 5    2160
dtype: int64
10800
(10800, 61)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [79]:
#predict without label 7 and 6
###exclude data labeled as 7 out from the testing
X_test_sub_OVA_5 = dataset_FE_t[cols_OVA[0:61]]
X_test_sub_OVA_5['ID']=pd.DataFrame({'ID':ID})

excl_list = Result_OVA.loc[Result_OVA['pred']!=7,'ID']
excl_list = list(excl_list.values)
tempset=set(excl_list) - set(list(Result_OVA_6.loc[Result_OVA_6['pred']==6,'ID'].values))

X_test_sub_OVA_5 = X_test_sub_OVA_5[X_test_sub_OVA_5['ID'].isin(list(tempset))]
X_test_sub_OVA_5.shape

#make prediction
pred_OVA_5 = best_model_GV_OVA5.predict(X_test_sub_OVA_5[cols_OVA[0:61]])

Result_OVA_5= pd.DataFrame(
    {'ID': X_test_sub_OVA_5['ID'].values,
     'pred': pred_OVA_5
    })
print Result_OVA_5.groupby('pred').size()

ID_5 = X_test_sub_OVA_5['ID'].values

print(ID_5.shape)

print(len(pred_OVA_5))

#write the prediction results out to a csv file for future consolidation
with open("Result_OVA_5.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(pred_OVA_5)):
        subfile.write("%s,%s\n"%(ID_5[i],pred))


pred
-1    499667
 5     20467
dtype: int64
(520134,)
520134


### Predict type 4

In [85]:
####train the model
dataset_OVA = dataset_FE[cols_OVA]
df_4 = dataset_OVA.loc[dataset_OVA['Cover_Type']!=7]
df_4 = df_4.loc[df_4['Cover_Type']!=6]
df_4 = df_4.loc[df_4['Cover_Type']!=5]
print df_4.groupby('Cover_Type').size()

df_4.loc[df_4['Cover_Type'] != 4, 'Cover_Type'] = -1
print df_4.groupby('Cover_Type').size()

X_train_OVA_4 = df_4.iloc[:,0:61].values
y_train_OVA_4 = df_4.iloc[:,61].values
print len(y_train_OVA_4)
print X_train_OVA_4.shape

###train without instances labeled as 7
best_model_GV_OVA4 = ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=-1)
best_model_GV_OVA4.fit(X_train_OVA_4,y_train_OVA_4)

Cover_Type
1    2160
2    2160
3    2160
4    2160
dtype: int64
Cover_Type
-1    6480
 4    2160
dtype: int64
8640
(8640, 61)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [86]:
#predict without label 7,6 and 5
###exclude data labeled as 7,6 and 5 out from the testing
X_test_sub_OVA_4 = dataset_FE_t[cols_OVA[0:61]]
X_test_sub_OVA_4['ID']=pd.DataFrame({'ID':ID})

excl_list = Result_OVA.loc[Result_OVA['pred']!=7,'ID']
excl_list = list(excl_list.values)
tempset=set(excl_list) - set(list(Result_OVA_6.loc[Result_OVA_6['pred']==6,'ID'].values))
tempset=tempset - set(list(Result_OVA_5.loc[Result_OVA_5['pred']==5,'ID'].values))

X_test_sub_OVA_4 = X_test_sub_OVA_4[X_test_sub_OVA_4['ID'].isin(list(tempset))]
print(X_test_sub_OVA_4.shape)

#make prediction
pred_OVA_4 = best_model_GV_OVA4.predict(X_test_sub_OVA_4[cols_OVA[0:61]])

Result_OVA_4= pd.DataFrame(
    {'ID': X_test_sub_OVA_4['ID'].values,
     'pred': pred_OVA_4
    })
print Result_OVA_4.groupby('pred').size()

ID_4 = X_test_sub_OVA_4['ID'].values

print(ID_4.shape)

print(len(pred_OVA_4))

#write the prediction results out to a csv file for future consolidation
with open("Result_OVA_4.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(pred_OVA_4)):
        subfile.write("%s,%s\n"%(ID_4[i],pred))

(499667, 62)
pred
-1    497686
 4      1981
dtype: int64
(499667,)
499667


### Predict type 1,2,3

In [87]:
####train the model with data excluding type 4,5, 6,7
dataset_OVA = dataset_FE[cols_OVA]
df_123 = dataset_OVA.loc[dataset_OVA['Cover_Type']!=7]
df_123 = df_123.loc[df_123['Cover_Type']!=6]
df_123 = df_123.loc[df_123['Cover_Type']!=5]
df_123 = df_123.loc[df_123['Cover_Type']!=4]
print df_123.groupby('Cover_Type').size()

X_train_OVA_123 = df_123.iloc[:,0:61].values
y_train_OVA_123 = df_123.iloc[:,61].values
print len(y_train_OVA_123)
print X_train_OVA_123.shape

###train without instances labeled as 7, 6,5,4
best_model_GV_OVA123 = ExtraTreesClassifier(n_estimators=150,random_state=42,n_jobs=-1)
best_model_GV_OVA123.fit(X_train_OVA_123,y_train_OVA_123)

Cover_Type
1    2160
2    2160
3    2160
dtype: int64
6480
(6480, 61)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [89]:
#predict without label 7 and 6, 5,4
###exclude data labeled as 7,6,5,4 out from the testing
X_test_sub_OVA_123 = dataset_FE_t[cols_OVA[0:61]]
X_test_sub_OVA_123['ID']=pd.DataFrame({'ID':ID})

excl_list = Result_OVA.loc[Result_OVA['pred']!=7,'ID']
excl_list = list(excl_list.values)
tempset=set(excl_list) - set(list(Result_OVA_6.loc[Result_OVA_6['pred']==6,'ID'].values))
tempset=tempset - set(list(Result_OVA_5.loc[Result_OVA_5['pred']==5,'ID'].values))
tempset=tempset - set(list(Result_OVA_4.loc[Result_OVA_4['pred']==4,'ID'].values))

X_test_sub_OVA_123 = X_test_sub_OVA_123[X_test_sub_OVA_123['ID'].isin(list(tempset))]
print(X_test_sub_OVA_123.shape)

#make prediction
pred_OVA_123 = best_model_GV_OVA123.predict(X_test_sub_OVA_123[cols_OVA[0:61]])

Result_OVA_123= pd.DataFrame(
    {'ID': X_test_sub_OVA_123['ID'].values,
     'pred': pred_OVA_123
    })
print Result_OVA_123.groupby('pred').size()

ID_123 = X_test_sub_OVA_123['ID'].values

print(ID_123.shape)

print(len(pred_OVA_123))

with open("Result_OVA_123.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i, pred in enumerate(list(pred_OVA_123)):
        subfile.write("%s,%s\n"%(ID_123[i],pred))

(497686, 62)
pred
1    212851
2    246188
3     38647
dtype: int64
(497686,)
497686


### Concatenate all together

In [90]:
frames = [Result_OVA_123, Result_OVA_4.loc[Result_OVA_4['pred']==4],Result_OVA_5.loc[Result_OVA_5['pred']==5],Result_OVA_6.loc[Result_OVA_6['pred']==6],Result_OVA.loc[Result_OVA['pred']==7]]

Result_OVA_ALL = pd.concat(frames)

Result_OVA_ALL.shape

(565892, 2)

In [95]:
#write data out for submission
with open("submission_OVA_all.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for index, row in Result_OVA_ALL.iterrows():
        subfile.write("%s,%s\n"%(row['ID'],row['pred']))

In [96]:
Result_OVA_ALL.groupby('pred').size()

pred
1    212851
2    246188
3     38647
4      1981
5     20467
6     19020
7     26738
dtype: int64